# Setup

In [ ]:
# Common imports
import sys
import os
import random
import sklearn
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import PIL

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_random_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib.pyplot as plt

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# Directories
localDir = 'images/lego/'

# Images
imgWidth = 64
imgHeight = 64

# Make images into multidimensional arrays

In [ ]:
def convertImageToArray(fileName):
    an_image = PIL.Image.open(fileName).convert('L')
    image_sequence = an_image.getdata()
    return np.array(image_sequence).reshape(imgWidth, imgHeight)

In [ ]:
IndexToBrickId = {}
brickIdImageKeyPairs = []

for index, key in enumerate(os.listdir(localDir)):
    print(key)
    IndexToBrickId[index] = key
    
    fileList = os.listdir(localDir + key)
    listLenStr = str(len(fileList))
    for index2, fileName in enumerate(fileList):
        filePath = localDir + key + '/' + fileName
        print(str(index2+1).zfill(len(listLenStr)) + '/' + listLenStr + ' - ' + filePath + (' ' * 256), end='\r')
        brickIdImageKeyPairs.append((index, convertImageToArray(filePath)))
    print(listLenStr + '/' + listLenStr + ' - Finished converting' + (' ' * 256))
print('\nFinished converting all')

# Randomize order

In [ ]:
brickIdIndexes = []
images = []

random.shuffle(brickIdImageKeyPairs)
for item in brickIdImageKeyPairs:
    brickIdIndexes.append(item[0])
    images.append(item[1])

In [ ]:
print(brickIdIndexes[0])
print(IndexToBrickId[0])
print(images[0] / 255)
print(len(images[0]))

# Machine Learning
Decision forest

In [ ]:
percentageToTakeTrainFull = int(len(brickIdIndexes) * 0.9)
print('Train ' + str(percentageToTakeTrainFull) + '/' + str(len(brickIdIndexes)))
X_train_full = np.asarray(images[:percentageToTakeTrainFull])
y_train_full = np.asarray(brickIdIndexes[:percentageToTakeTrainFull])
X_test = np.asarray(images[percentageToTakeTrainFull:])
y_test = np.asarray(brickIdIndexes[percentageToTakeTrainFull:])

In [ ]:
# Show the size and dimension of the dataset.
X_train_full.shape

In [ ]:
# Each pixel intensity is represented as a byte (0 to 255).
X_train_full.dtype

In [ ]:
percentageToTakeForValidation = len(X_test)
print('Validate ' + str(percentageToTakeForValidation) + '/' + str(len(X_train_full)))

# Split the full training set into a validation set and a (smaller) training set,
# and scale the pixel intensities down to the 0-1 range and convert them to floats, by dividing by 255.
X_valid, X_train = X_train_full[:percentageToTakeForValidation] / 255., X_train_full[percentageToTakeForValidation:] / 255.
y_valid, y_train = y_train_full[:percentageToTakeForValidation], y_train_full[percentageToTakeForValidation:]
X_test = X_test / 255.

In [ ]:
X_train.shape

In [ ]:
X_valid.shape

In [ ]:
X_test.shape

# Show random image and class

In [ ]:
#Plot an image using Matplotlib's imshow() function, with a gray color map:
plt.imshow(X_train[0], cmap='gray')
plt.show()

In [ ]:
# Show the name of the first image in the training set.
IndexToBrickId[y_train[0]]

## Standardize the data
Because we want to use the SELU activation function and LeCun weight initializer, we should standardize all the input features to a mean of 0 and a standard deviation of 1. Since each pixel is an input feature, there are 28x28=784 input features, and we need to compute the mean and standard deviation for each of them.

In [ ]:
# Compute the mean for each pixel.
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_means.shape

In [ ]:
# Compute the standard deviation for each pixel.
pixel_stds = X_train.std(axis=0, keepdims=True)
pixel_stds.shape

In [ ]:
# Scale the inputs to mean 0 and standard deviation 1 to achieve self-normalization with SELU and LeCun.
X_train_standardized = (X_train - pixel_means) / pixel_stds
X_valid_standardized = (X_valid - pixel_means) / pixel_stds
X_test_standardized = (X_test - pixel_means) / pixel_stds

In [ ]:
# Validate that the mean is close to 0 for each pixel.
X_train_standardized.mean(axis=0, keepdims=True)

In [ ]:
# Validate that the standard deviation is close to 1 for each pixel.
X_train_standardized.std(axis=0, keepdims=True)

# Create a model using the Sequential API

In [ ]:
model = keras.models.Sequential([
# Input layer:
# A "Flatten" layer converts each input image into a 1-dimensional array.
keras.layers.Flatten(input_shape=[imgWidth, imgHeight]),

# Hidden layers:
# A dense layer is fully connected.
keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),

# Output layer.
# The layer contains one neuron per class (i.e. 10).
# Since it is multiclass classification, we should use the softmax activation function.
# It will ensure that the estimated probabilities are between 0 and 1, and that the sum
# of estimated probabilities for one prediction is 1.
# (for binary classification we would have a single output neuron using the logistic activation function).
keras.layers.Dense(10, activation="softmax")])

### Show information about the model

In [ ]:
model.summary()

In [ ]:
hidden1 = model.layers[1]
hidden1.name

In [ ]:
weights, biases = hidden1.get_weights()

In [ ]:
weights

In [ ]:
weights.shape

In [ ]:
biases

In [ ]:
biases.shape

## Compile the model
You must at least specify the loss function and the optimizer to use. You can also specify a list of additional metrics to use during training and evaluation.

In [ ]:
# "sparse_categorical_crossentropy" is the loss function to use for classification when the classes are exclusive.
# "sgd" means Stochastic Gradient Descent.
# "accuracy" enables us to measure the accuracy during training and evaluation.
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(momentum=0.9),
              metrics=["accuracy"])

## Train the model

In [ ]:
# EarlyStopping (with rollback to the best model).
early_stopping = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

# Performance scheduling
# (multiply the learning rate by a factor when the error stops dropping for a number of epochs, called patience)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=4)

# Train the model with early stopping or performance scheduling or both. Training is much faster when
# early stopping is used, but a slightly better accuracy is achieved with performance scheduling alone.
history = model.fit(X_train_standardized, y_train, epochs=30,
                    validation_data=(X_valid_standardized, y_valid),
                    callbacks=[lr_scheduler, early_stopping])

In [ ]:
# The fit() method returns a history object with information about the result of the training.
history.params

In [ ]:
print(history.epoch)

In [ ]:
history.history.keys()

In [ ]:
# Show the learning curves.
# (The training curves should be shifted half an epoch to the left to be completely comparable with
# the validation curves).

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

## Evaluate the model.

In [ ]:
model.evaluate(X_test_standardized, y_test)

In [ ]:
# Make predictions with probabilities for the first 3 instances in the test set.
X_new = X_test_standardized[:3]
y_proba = model.predict(X_new)
y_proba.round(2)

In [ ]:
# Make predictions without probabilities.
y_pred = model.predict_classes(X_new)
y_pred

In [ ]:
# Check if the predictions were coorrect.
y_new = y_test[:3]
y_new

In [ ]:
# Compare image and classes.
newClassList = np.array(list(IndexToBrickId.values()))[y_pred]

for i in range(3):
    plt.imshow(X_new[i], cmap='gray')
    plt.show()
    print(newClassList[i])